Run the next cells in the Python REPL inside the container, e.g. after

`docker run -it [your_image] /bin/bash`

> Tip : if you get disconnected, get the container-id from a `docker ps` and 

`docker exec -it <container-id> bash`

In [ ]:
import logging
import boto3
from botocore.exceptions import ClientError
import os
import getpass # Not allowed on SSC DataHub

Testing uploading a single file on S3 manually

In [ ]:
ACCESS_KEY = getpass.getpass(prompt='ACCESS_KEY: ')
SECRET_KEY = getpass.getpass(prompt='SECRET_KEY: ')

# Using an existing S3 bucket for tests
bucket_name = "eccc-msc-bom-out"

client = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY,
    aws_secret_access_key=SECRET_KEY
)

# Slightly modified from https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-uploading-files.html#uploading-files

def upload_file(client, file_name, bucket_name, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket_name: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    try:
        response = client.upload_file(file_name, bucket_name, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [ ]:
# `client` must be configured properly; if you edited the 'keys' cell above and re-ran it, you're toast :-)

# >>> os.getcwd() # WORKDIR in container as per Dockerfile
# '/sarra'
# >>> filename = "requirements.txt" # test uploading that file
upload_file(client, filename, bucket_name, object_name="Test_file.txt")

Testing uploading a single file on S3 via `metpx-cloud-publisher.publish_to_s3`. No `sr_subscribe` yet.

In [ ]:
# `import metpx-cloud-publisher` will not work because [hyphens](https://stackoverflow.com/questions/65930543/importing-a-python-custom-package-with-hyphen)
#import importlib

importlib.import_module('metpx-cloud-publisher') # will not work
# Traceback (most recent call last):
# ...
# File "/sarra/metpx-cloud-publisher.py", line 154, in <module>
#    event = MetPXCloudPublisher(self)  # noqa
# NameError: name 'self' is not defined
